# Boolean Search in Documents

## Objective
Expand the simple term search functionality to include Boolean search capabilities. This will allow users to perform more complex queries by combining multiple search terms using Boolean operators.

## Problem Description
You must enhance the existing search engine from the previous exercise to support Boolean operators: AND, OR, and NOT. This will enable the retrieval of documents based on the logical relationships between multiple terms.

## Requirements

### Step 1: Update Data Preparation
Ensure that the documents are still loaded and preprocessed from the previous task. The data should be clean and ready for advanced querying.

In [1]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import SnowballStemmer
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VELA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VELA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Step 2: Create an Inverted Index

Create an inverted index from the documents. This index maps each word to the set of document IDs in which that word appears. This facilitates word lookup in the search process.

In [2]:
def create_inverted_index(directory):
    index = {}
    stemmer = SnowballStemmer('english')
    
    for root, dirs, files in os.walk(directory):
        for file in files[:100]:  
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read().lower()  
                tokens = word_tokenize(content)
                tokens = [stemmer.stem(token) for token in tokens if token not in stopwords.words('english') and token not in string.punctuation]
                for word in tokens:
                    if word in index:
                        index[word].append(file_path)
                    else:
                        index[word] = [file_path]
    return index

In [3]:
directory = "../libros"

inverted_index = create_inverted_index(directory)
print("Inverted index created.")

Inverted index created.


In [4]:
# Convert the inverted index into a dataframe
df_index = pd.DataFrame([(word, files) for word, files in inverted_index.items()], columns=['Word', 'Files'])

# Save the inverted index as a CSV file
df_index.to_csv('inverted_index.csv', index=False)

print("Inverted index saved as 'inverted_index.csv'.")

Inverted index saved as 'inverted_index.csv'.


In [5]:
# Cargar el archivo CSV generado
df_index = pd.read_csv('inverted_index.csv')

# Mostrar el DataFrame cargado
print(df_index.head())

        Word                                              Files
0       ﻿the  ['../libros\\pg100.txt', '../libros\\pg100.txt...
1    project  ['../libros\\pg100.txt', '../libros\\pg100.txt...
2  gutenberg  ['../libros\\pg100.txt', '../libros\\pg100.txt...
3      ebook  ['../libros\\pg100.txt', '../libros\\pg100.txt...
4    complet  ['../libros\\pg100.txt', '../libros\\pg100.txt...


### Step 3: Implementing Boolean Search
- **Enhance Input Query**: Modify the function to accept complex queries that can include the Boolean operators AND, OR, and NOT.
- **Implement Boolean Logic**:
  - **AND**: The document must contain all the terms. For example, `python AND programming` should return documents containing both "python" and "programming".
  - **OR**: The document can contain any of the terms. For example, `python OR programming` should return documents containing either "python", "programming", or both.
  - **NOT**: The document must not contain the term following NOT. For example, `python NOT snake` should return documents that contain "python" but not "snake".

### Step 4: Query Processing
- **Parse the Query**: Implement a function to parse the input query to identify the terms and operators.
- **Search Documents**: Based on the parsed query, implement the logic to retrieve and rank the documents according to the Boolean expressions.
- **Handling Case Sensitivity and Partial Matches**: Optionally, you can handle cases and partial matches to refine the search results.

In [6]:
def process_query(query, inverted_index):
    query_parts = query.split()
    result = None
    
    while query_parts:
        token = query_parts.pop(0)
        if token == 'AND':
            operand1 = result
            operand2 = set(inverted_index[query_parts.pop(0)])
            result = operand1.intersection(operand2)
        elif token == 'OR':
            operand1 = result
            operand2 = set(inverted_index[query_parts.pop(0)])
            result = operand1.union(operand2)
        elif token == 'NOT':
            operand1 = result
            operand2 = set(inverted_index[query_parts.pop(0)])
            result = operand1.difference(operand2)
        else:
            result = set(inverted_index[token])
    
    return result

### Step 5: Displaying Results
- **Output the Results**: Display the documents that match the query criteria. Include functionalities to handle queries that result in no matching documents.

In [8]:
query = "hello"
result = process_query(query, inverted_index)

# Formatear y mostrar resultados de manera más elegante
formatted_results = [os.path.basename(file_path) for file_path in result]
print("Documentos encontrados:")
for file_name in formatted_results:
    print(file_name)

Documentos encontrados:
pg10676.txt
pg4300.txt
pg74.txt
pg76.txt
pg73441.txt
pg55.txt
pg64317.txt
pg408.txt
pg39407.txt
pg19926.txt
pg2814.txt
pg1184.txt


## Evaluation Criteria
- **Correctness**: The Boolean search implementation should correctly interpret and process the queries according to the Boolean logic.
- **Efficiency**: Consider the efficiency of your search process, especially as the complexity of queries increases.
- **User Experience**: Ensure that the interface for inputting queries and viewing results is user-friendly.

## Additional Challenges (Optional)
- **Nested Boolean Queries**: Allow for nested queries using parentheses, such as `(python OR java) AND programming`.
- **Phrase Searching**: Implement the ability to search for exact phrases enclosed in quotes.
- **Proximity Searching**: Extend the search to find terms that are within a specific distance from one another.

This exercise will deepen your understanding of how search engines process and respond to complex user queries. By incorporating Boolean search, you not only enhance the functionality of your search engine but also mimic more closely how real-world information retrieval systems operate.